## Preprocessing

In [1]:
pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.1 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at STATUS value counts to determine usefulness of data
application_df["STATUS"].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [6]:
#Remove Rows that have a 0 for STATUS
application_df= application_df[application_df['STATUS']!=0]

application_df["STATUS"].value_counts()

1    34294
Name: STATUS, dtype: int64

In [7]:
#Remove STATUS column due to lack of usefulness
application_df = application_df.drop(columns="STATUS")

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [8]:
# Look at SPECIAL_CONSIDERATIONS value counts to determine usefulness of data
application_df["SPECIAL_CONSIDERATIONS"].value_counts()

N    34268
Y       26
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [9]:
#Remove the data where special considerations = Y
application_df=application_df[application_df["SPECIAL_CONSIDERATIONS"] !='Y']

application_df["SPECIAL_CONSIDERATIONS"].value_counts()

N    34268
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [10]:
#Remove SPECIAL CONSIDERATIONS column due to lack of usefulness
application_df = application_df.drop(columns="SPECIAL_CONSIDERATIONS")

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [11]:
# Look at AFFILIATION value counts for binning
application_df["AFFILIATION"].value_counts()

Independent         18451
CompanySponsored    15703
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of affiliations to be replaced, less than <1% of the data
affiliation_to_replace= ["Family/Parent", "National", "Regional", "Other"]

# Replace in dataframe
for aff in affiliation_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(aff,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18451
CompanySponsored    15703
Other                 114
Name: AFFILIATION, dtype: int64

In [13]:
# Look at INCOME_AMT value counts for binning
application_df["INCOME_AMT"].value_counts()

0                24370
25000-99999       3745
100000-499999     3373
1M-5M              955
1-9999             721
10000-24999        540
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [14]:
# Rather than using a cutoff value and an other category for binning since the data is already binned.
#It will be regrouped into a "No_Income" (0), "Lower_Income" (1-1M) and "Higher_Income" (1M+)
no_income_to_replace= ["0"]

# Replace in dataframe
for inc in no_income_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(inc,"No_Income")

low_income_to_replace= ["1-9999","10000-24999","25000-99999", "100000-499999"]

# Replace in dataframe
for inc in low_income_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(inc,"Lower_Income")

high_income_to_replace= ["1M-5M","5M-10M","10M-50M", "50M+"]

# Replace in dataframe
for inc in high_income_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(inc,"Higher_Income")
# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

No_Income        24370
Lower_Income      8379
Higher_Income     1519
Name: INCOME_AMT, dtype: int64

In [15]:
# Look at ORGANIZATION value counts for binning
application_df["ORGANIZATION"].value_counts()

Trust           23491
Association     10248
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [16]:
# Look at USE_CASE value counts for binning
application_df["USE_CASE"].value_counts()

Preservation     28066
ProductDev        5671
CommunityServ      382
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [17]:
# Choose a cutoff value and create a list of affiliations to be replaced, less than <1% of the data
use_case_to_replace= ["Heathcare", "Other"]

# Replace in dataframe
for use in use_case_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(use,"Other")

# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

Preservation     28066
ProductDev        5671
CommunityServ      382
Other              149
Name: USE_CASE, dtype: int64

In [18]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27006
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`.  I chose any value that would be 1% or less of the data (343) as my cutoff value.
application_types_to_replace= ["T17", "T15", "T29", "T14", "T25", "T2", "T12", "T13", "T9"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27006
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17307
C2000     6069
C1200     4833
C3000     1918
C2100     1882
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [21]:
# You may find it helpful to look at CLASSIFICATION value counts >1
filtered_classification = application_df["CLASSIFICATION"].value_counts()

filtered_classification = filtered_classification[filtered_classification >1]

filtered_classification

C1000    17307
C2000     6069
C1200     4833
C3000     1918
C2100     1882
C7000      776
C1700      286
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [23]:
#Get a list of the classifications to replace
replacing_classification = application_df["CLASSIFICATION"].value_counts()

replacing_classification = replacing_classification[replacing_classification < 1000]

classification_list = replacing_classification.index.tolist()

classification_list

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [24]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_list

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17307
C2000     6069
C1200     4833
Other     2259
C3000     1918
C2100     1882
Name: CLASSIFICATION, dtype: int64

In [25]:
# Convert categorical data to numeric with `pd.get_dummies`
df_encoded = pd.get_dummies(application_df[["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT"]])

application_df_encoded = pd.concat([application_df, df_encoded], axis=1)


In [26]:
# Split our preprocessed data into our features and target arrays
y=application_df_encoded["IS_SUCCESSFUL"]
X=application_df_encoded.drop(columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT","IS_SUCCESSFUL"])



In [27]:
#Review the X data
X.head()

,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_Higher_Income,INCOME_AMT_Lower_Income,INCOME_AMT_No_Income
0,5000,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
1,108590,0,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
2,5000,0,0,0,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,1
3,6692,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,142590,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0


In [28]:
# Check the balance of our target values
y.value_counts()

1    18242
0    16026
Name: IS_SUCCESSFUL, dtype: int64

In [29]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation="relu", input_dim=30))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                1860      
                                                                 
 dense_1 (Dense)             (None, 10)                610       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2481 (9.69 KB)
Trainable params: 2481 (9.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Train the model
fit_model=nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5835 - accuracy: 0.7065
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5602 - accuracy: 0.7255
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5574 - accuracy: 0.7284
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7274
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5551 - accuracy: 0.7292
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5546 - accuracy: 0.7289
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5540 - accuracy: 0.7287
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7294
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7287
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5526 - accura

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5404 - accuracy: 0.7384 - 509ms/epoch - 2ms/step
Loss: 0.5403873920440674, Accuracy: 0.7384148240089417


Automated Hyperparameter Model Optimization

In [35]:
#Create a function
def create_model(hp):
  nn_model = tf.keras.models.Sequential()
  activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

  nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units', min_value = 1, max_value=30, step=5),
                                     activation=activation, input_dim=30))

  for i in range(hp.Int('num_layers', 1, 6)):
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_'+str(i), min_value=1, max_value=30, step=5), activation=activation))

  nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

  nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

  return nn_model

In [36]:
import keras_tuner as kt

tuner=kt.Hyperband(create_model, objective="val_accuracy", max_epochs=20, hyperband_iterations=2)

In [37]:
#Run for the best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 60 Complete [00h 00m 53s]
val_accuracy: 0.7361970543861389

Best val_accuracy So Far: 0.7394654154777527
Total elapsed time: 00h 24m 16s


In [38]:
#Get best hyperparameterers
best_hyper = tuner.get_best_hyperparameters(1)[0]

best_hyper.values

{'activation': 'relu',
 'first_units': 16,
 'num_layers': 4,
 'units_0': 11,
 'units_1': 1,
 'units_2': 6,
 'units_3': 6,
 'units_4': 1,
 'units_5': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [39]:
#Evaluate on data
best_model = tuner.get_best_models(1)[0]

model_loss, model_accuracy=best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5438 - accuracy: 0.7395 - 585ms/epoch - 2ms/step
Loss: 0.5438134074211121, Accuracy: 0.7394654154777527


In [40]:
model_3 = tf.keras.models.Sequential()

# First layer
model_3.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=30))

# First hidden layer
model_3.add(tf.keras.layers.Dense(units=11, activation="relu"))

# Second hidden layer
model_3.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Third hidden layer
model_3.add(tf.keras.layers.Dense(units=6, activation="relu"))

# Fourth hidden layer
model_3.add(tf.keras.layers.Dense(units=6, activation="relu"))

# Fifth hidden layer
model_3.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Output layer
model_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

model_3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                496       
                                                                 
 dense_7 (Dense)             (None, 11)                187       
                                                                 
 dense_8 (Dense)             (None, 1)                 12        
                                                                 
 dense_9 (Dense)             (None, 6)                 12        
                                                                 
 dense_10 (Dense)            (None, 6)                 42        
                                                                 
 dense_11 (Dense)            (None, 1)                 7         
                                                                 
 dense_12 (Dense)            (None, 1)                

In [41]:
model_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [42]:
# Train the model
fit_model3=model_3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6247 - accuracy: 0.6848
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5858 - accuracy: 0.7250
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5782 - accuracy: 0.7265
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5752 - accuracy: 0.7272
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5724 - accuracy: 0.7273
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5711 - accuracy: 0.7280
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5706 - accuracy: 0.7280
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5685 - accuracy: 0.7288
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5669 - accuracy: 0.7281
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5663 - accura

In [43]:
model_loss, model_accuracy = model_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5533 - accuracy: 0.7362 - 529ms/epoch - 2ms/step
Loss: 0.5533460974693298, Accuracy: 0.7361970543861389


Export Models

In [44]:
# Export our model to HDF5 file
from tensorflow.keras.models import save_model

hdf5_filename1 = "AlphabetSoupCharity_Optimization1.h5"

save_model(nn, hdf5_filename1)

print(f"Model saved to {hdf5_filename1}")

hdf5_filename2 = "AlphabetSoupCharity_Optimization2.h5"

save_model(best_model, hdf5_filename2)

print(f"Model saved to {hdf5_filename2}")

hdf5_filename3 = "AlphabetSoupCharity_Optimization3.h5"

save_model(model_3, hdf5_filename3)

print(f"Model saved to {hdf5_filename3}")

Model saved to AlphabetSoupCharity_Optimization1.h5
Model saved to AlphabetSoupCharity_Optimization2.h5
Model saved to AlphabetSoupCharity_Optimization3.h5


<ipython-input-44-4b48b5f11015>:6: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(nn, hdf5_filename1)
<ipython-input-44-4b48b5f11015>:12: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(best_model, hdf5_filename2)
<ipython-input-44-4b48b5f11015>:18: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model_3, hdf5_filename3)
